In [1]:
from torch.autograd import Variable
import torch
import pickle
import os
import sys
import matplotlib.pyplot as plt
import numpy as np
import evaluation_vectorization as ev
sys.path.append(os.path.join(os.getcwd(), 'Aging_Model'))

# Device

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

# Dataset

In [3]:
ds = 'Pendigits'

In [4]:
datapath = os.path.join(os.getcwd(), 'Datasets', 'Haibin',
                        'data_processed', f'Dataset_{ds}.p')
with open(datapath, 'rb') as f:
    dataset = pickle.load(f)
X = dataset['X'].float()
y = dataset['y']
E, N_features, N_class = X.shape[0], X.shape[1], torch.max(
    torch.unique(y)).item()+1
X.shape, y.shape, E, N_features, N_class

(torch.Size([10992, 16]), torch.Size([10992]), 10992, 16, 10)

In [5]:
# normalization
X = X / (torch.max(X, axis=0)[0] - torch.min(X, axis=0)[0])
X = X - torch.min(X, axis=0)[0]
torch.min(X), torch.max(X)

(tensor(0.), tensor(1.))

In [6]:
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
from torch.utils.data import random_split

# generate tensordataset
dataset = TensorDataset(X.to(device), y.to(device))

# split
train_rate = 0.6
test_rate = 0.2
valid_rate = 0.2
E_train = min(8192, int(E*train_rate))
E_test = min(2048, int(E*test_rate))
E_valid = min(2048, int(E*valid_rate))

train_data, rest_data = random_split(
    dataset, [E_train, E-E_train], generator=torch.Generator().manual_seed(19950102))
test_data, rest_data = random_split(rest_data, [
                                     E_test, E-E_train-E_test], generator=torch.Generator().manual_seed(19950102))
valid_data, rest_data = random_split(rest_data, [
                                     E_valid, E-E_train-E_test-E_valid], generator=torch.Generator().manual_seed(19950102))

len(train_data), len(test_data), len(valid_data)

(6595, 2048, 2048)

In [7]:
# batch
train_loader = DataLoader(train_data, batch_size=len(train_data))
test_loader = DataLoader(test_data, batch_size=len(test_data))
valid_loader = DataLoader(valid_data, batch_size=len(valid_data))

In [8]:
M_valid = 500
K_valid = 50

# Evaluation

## Load Models

In [9]:
with open(f'./result/{ds}_NN.p', 'rb') as f:
    NN = pickle.load(f)
NN.to(device)

Sequential(
  (0): Linear(in_features=16, out_features=10, bias=True)
  (1): Tanh()
  (2): Linear(in_features=10, out_features=10, bias=True)
  (3): Tanh()
)

In [10]:
with open(f'./result/{ds}_PNN.p', 'rb') as f:
    PNN = pickle.load(f)
PNN.to(device)

Sequential(
  (0): PNNLayer()
  (1): PNNLayer()
)

In [11]:
with open(f'./result/{ds}_AAPNN.p', 'rb') as f:
    AAPNN = pickle.load(f)
AAPNN.to(device)

Sequential(
  (0): PNNLayer()
  (1): PNNLayer()
)

# Get Structure

In [12]:
structure_NN = ev.GetStructure(NN)
structure_PNN = ev.GetStructure(PNN)
structure_AAPNN = ev.GetStructure(AAPNN)
structure_NN, structure_PNN, structure_AAPNN

('_16_10_10', '_16_10_10', '_16_10_10')

In [13]:
if (structure_NN == structure_PNN) and (structure_NN == structure_AAPNN):
    structure = structure_NN
structure

'_16_10_10'

## Evaluation

In [14]:
for x_valid, y_valid in valid_loader:
    prediction_valid = NN(x_valid)
    p = torch.argmax(prediction_valid, 1)
    pred_y = p.cpu().data.numpy().squeeze()
    acc_valid = sum(pred_y == y_valid.cpu().numpy()) / y_valid.shape[0]

np.savetxt(f'./result/{ds}_NN{structure}.txt', acc_valid.reshape(1,1))
acc_valid

0.9794921875

In [15]:
mean_acc_PNN, std_acc_PNN, mean_maa_PNN, std_maa_PNN = ev.Evaluation(PNN, valid_loader, M_valid, 1, K_valid, device)

  0%|          | 0/500 [00:00<?, ?it/s]

In [ ]:
mean_acc_AAPNN, std_acc_AAPNN, mean_maa_AAPNN, std_maa_AAPNN = ev.Evaluation(AAPNN, valid_loader, M_valid, 1, K_valid, device)

  0%|          | 0/500 [00:00<?, ?it/s]

# Visualization

In [ ]:
plt.fill_between(np.linspace(0,1,50), mean_acc_PNN-std_acc_PNN, mean_acc_PNN+std_acc_PNN, alpha=0.3, color='blue');
plt.fill_between(np.linspace(0,1,50), mean_acc_AAPNN-std_acc_AAPNN, mean_acc_AAPNN+std_acc_AAPNN, alpha=0.3, color='red');

plt.plot(np.linspace(0,1,50), mean_acc_PNN, label='PNN', color='blue');
plt.plot(np.linspace(0,1,50), mean_acc_AAPNN, label='AAPNN', color='red');

plt.xlabel('Normalized time');
plt.ylabel('Basic accuracy');
plt.xlim([0, 1]);
plt.legend();
plt.savefig(f'./result/{ds}_ACC{structure}.pdf', format='pdf')

In [ ]:
plt.fill_between(np.linspace(0,1,50), mean_maa_PNN-std_maa_PNN, mean_maa_PNN+std_maa_PNN, alpha=0.3, color='blue');
plt.fill_between(np.linspace(0,1,50), mean_maa_AAPNN-std_maa_AAPNN, mean_maa_AAPNN+std_maa_AAPNN, alpha=0.3, color='red');

plt.plot(np.linspace(0,1,50), mean_maa_PNN, label='PNN', color='blue');
plt.plot(np.linspace(0,1,50), mean_maa_AAPNN, label='AAPNN', color='red');

plt.xlabel('Normalized time');
plt.ylabel('Measureing aware accuracy');
plt.xlim([0, 1]);
plt.legend();
plt.savefig(f'./result/{ds}_MAA{structure}.pdf', format='pdf')